In [2]:
import os
import sys
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold

In [48]:
DATA_DIR = "/data2/mhassan/Pharos"
pname = "Caspase_1"
classifier_loglevel = 0
gridsearch_loglevel = 2

In [20]:
print("READING ACTIVES AND DECOYS")
actives = pd.read_csv(os.path.join(DATA_DIR, 'actives/fingerprints/Newdata_set2/'+pname+'.csv'), header=None)
decoys = pd.read_csv(os.path.join(DATA_DIR, 'decoys/fingerprints/Newdata_set2/decoys_'+pname+'.csv'), header=None)

print("Number of actives: {}, and number of decoys: {}".format(len(actives.index), len(decoys.index)))

READING ACTIVES AND DECOYS
Number of actives: 372, and number of decoys: 13356


In [22]:
actives.head()

,0,1,2,3,4,5,6,7,8,9,...,2683,2684,2685,2686,2687,2688,2689,2690,2691,2692
0,CC(C(=O)OCC(=O)[C@H](CC(=O)O)NC(=O)OCc1ccccc1)...,28,87,0,0,0,0,91,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CC(C)c1ccc(Oc2ccccc2S(=O)(=O)N[C@H](C=O)C(=O)O...,16,42,7,0,0,0,41,6,0,...,0,0,0,0,0,0,0,0,0,0
2,CCC(C)c1ccc(Oc2ccccc2S(=O)(=O)N[C@H](C=O)C(=O)...,16,42,7,0,0,0,41,6,0,...,0,0,0,0,0,0,0,0,0,0
3,CCCCC1=CC=C(NC(=O)CCc2ccccc2)C(=O)N1CC(=O)N[C@...,16,38,7,3,0,0,36,4,2,...,0,0,0,0,0,0,0,0,0,0
4,Cc1cccc(Oc2ccccc2S(=O)(=O)N[C@H](C=O)C(=O)O)c1C,16,42,7,0,0,0,36,6,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# Train the protein model only if there are more than 50 samples
if len(actives.index) < 50:
    exit()

# Start training
print("TRAINING FOR ", pname)

# Get the score and output path, if not found then create one
score_path = os.path.join(DATA_DIR, "scores/part2")
if not os.path.isdir(score_path):
    os.makedirs(score_path)
output = open(os.path.join(score_path, pname+'_score.csv'),'w')

actives_x = actives.iloc[:,1:].values
actives_y = np.ones(len(actives_x))

decoys_x = decoys.iloc[:,:].values
decoys_y = np.zeros(len(decoys_x))

print("PROCESSING TRAINING DATA")
x = np.concatenate((actives_x, decoys_x))
y = np.concatenate((actives_y, decoys_y)) #labels

# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state =1)

TRAINING FOR  Caspase_1
PROCESSING TRAINING DATA


In [43]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

In [49]:
print("BUILING CLASSIFIERS")

# Scoring metrics
scoring = {'auc_score': 'roc_auc', 
           'precision_score': make_scorer(precision_score),
           'recall_score': make_scorer(recall_score),
           'accuracy_score': make_scorer(accuracy_score)
          }

# Stratified K-fold
cross_validation_count = 5
skf = StratifiedKFold(n_splits=cross_validation_count)

# Random Forest
classifier_rf = RandomForestClassifier(class_weight='balanced', verbose=classifier_loglevel)
parameters_rf = {'n_estimators':[i for i in range(100, 1000, 50)]}
gridsearch_rf = GridSearchCV(classifier_rf, parameters_rf, scoring=scoring, cv=skf, refit='auc_score', n_jobs=-1, verbose=gridsearch_loglevel)
print("TRAINING THE RANDOM FOREST CLASSIFIER")
gridsearch_rf.fit(x_train, y_train)

# Support Vector Machine
classifier_sv = SVC(class_weight='balanced', kernel='linear', random_state=1, verbose=classifier_loglevel)
parameters_sv = {'C': [0.1, 1.0, 10, 100, 1000], 'gamma':[0.1, 1, 10, 100, 1000, 'auto']}
gridsearch_sv = GridSearchCV(classifier_sv, parameters_sv, scoring=scoring, cv=skf, refit='auc_score', n_jobs=-1, verbose=gridsearch_loglevel)
print("TRAINING THE SUPPORT VECTOR CLASSIFIER")
gridsearch_sv.fit(x_train, y_train)

# Neural Network
def get_hidden_layers():
    import itertools
    x = [64, 128, 256, 512]
    hl = []
    
    for i in range(1, len(x)):
        hl.extend([p for p in itertools.product(x, repeat=i+1)])
    
    return hl

classifier_nn = MLPClassifier(solver='adam', alpha=1e-5, early_stopping=True, random_state=1, verbose=classifier_loglevel)
hidden_layer_sizes = get_hidden_layers()
parameters_nn = {'hidden_layer_sizes': hidden_layer_sizes}
gridsearch_nn = GridSearchCV(classifier_nn, parameters_nn, scoring=scoring, cv=skf, refit='auc_score', n_jobs=-1, verbose=gridsearch_loglevel)
print("TRAINING THE NEURAL NETWORK CLASSIFIER")
gridsearch_nn.fit(x_train, y_train)

# print("SAVING CLASSIFIERS")
# if sv_clf.best_score_ > rf_clf.best_score_:
#     model = sv_clf.best_estimator_
#     joblib.dump(model, os.path.join(DATA_DIR, 'models/part2/'+pname))
#     output.write(pname+','+str(sv_clf.best_score_)+','+str(sv_clf.best_params_)+'\n')
#     output.close()
# else:
#     model = rf_clf.best_estimator_
#     joblib.dump(model, os.path.join(DATA_DIR, 'models/part2/'+pname))
#     output.write(pname+','+str(rf_clf.best_score_)+','+str(rf_clf.best_params_)+'\n')
#     output.close()

BUILING CLASSIFIERS
Training the Random Forest Classifier
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=150 ................................................
[CV] n_estimators=150 ................................................
[CV] n_estimators=150 ................................................
[CV] n_estimators=150 ................................................
[CV] n_estimators=150 ................................................
[CV] ................................. n_estimators=100, total=   3.2s
[CV] n_estimators=200 ................................................
[CV] n_estimators=200 .......

[Parallel(n_jobs=-1)]: Done  25 out of  90 | elapsed:   23.4s remaining:  1.0min


[CV] n_estimators=600 ................................................
[CV] n_estimators=650 ................................................
[CV] n_estimators=650 ................................................
[CV] n_estimators=650 ................................................
[CV] n_estimators=650 ................................................
[CV] n_estimators=650 ................................................
[CV] n_estimators=700 ................................................
[CV] ................................. n_estimators=350, total=  12.0s
[CV] n_estimators=700 ................................................
[CV] n_estimators=700 ................................................
[CV] ................................. n_estimators=350, total=  11.5s
[CV] n_estimators=700 ................................................
[CV] n_estimators=700 ................................................
[CV] ................................. n_estimators=350, total=  13.0s
[CV] n

[Parallel(n_jobs=-1)]: Done  71 out of  90 | elapsed:  1.3min remaining:   20.6s


[CV] ................................. n_estimators=750, total=  40.9s
[CV] ................................. n_estimators=850, total=  37.0s
[CV] ................................. n_estimators=850, total=  37.1s
[CV] ................................. n_estimators=850, total=  37.1s
[CV] ................................. n_estimators=800, total=  40.8s
[CV] ................................. n_estimators=900, total=  35.8s
[CV] ................................. n_estimators=750, total=  45.1s
[CV] ................................. n_estimators=800, total=  43.7s
[CV] ................................. n_estimators=900, total=  39.0s
[CV] ................................. n_estimators=850, total=  40.4s
[CV] ................................. n_estimators=850, total=  41.7s
[CV] ................................. n_estimators=900, total=  40.2s
[CV] ................................. n_estimators=900, total=  40.3s
[CV] ................................. n_estimators=950, total=  41.1s
[CV] .

...

[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:  4.1min


[CV] hidden_layer_sizes=(64, 64, 128) ................................
[CV] hidden_layer_sizes=(64, 64, 128) ................................
[CV] .................... hidden_layer_sizes=(256, 256), total= 1.9min
[CV] hidden_layer_sizes=(64, 64, 128) ................................
[CV] hidden_layer_sizes=(64, 64, 128) ................................
[CV] .................... hidden_layer_sizes=(256, 256), total= 2.3min
[CV] hidden_layer_sizes=(64, 64, 128) ................................
[CV] ..................... hidden_layer_sizes=(256, 64), total= 2.8min
[CV] hidden_layer_sizes=(64, 64, 256) ................................
[CV] .................... hidden_layer_sizes=(256, 256), total= 2.2min
[CV] hidden_layer_sizes=(64, 64, 256) ................................
[CV] hidden_layer_sizes=(64, 64, 256) ................................
[CV] hidden_layer_sizes=(64, 64, 256) ................................
[CV] .................... hidden_layer_sizes=(256, 512), total= 2.2min
[CV] h

Process ForkPoolWorker-281:
Process ForkPoolWorker-321:
Process ForkPoolWorker-286:
Process ForkPoolWorker-323:
Process ForkPoolWorker-336:
Process ForkPoolWorker-334:
Process ForkPoolWorker-331:
Process ForkPoolWorker-329:
Process ForkPoolWorker-332:
Process ForkPoolWorker-330:
Process ForkPoolWorker-333:
Process ForkPoolWorker-335:
Process ForkPoolWorker-328:
Traceback (most recent call last):
/home/mhassan/anaconda3/envs/htmd/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
Traceback (most recent call last):
Process ForkPoolWorker-327:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
/home/mhassan/anaconda3/envs/htmd/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWa

  File "/home/mhassan/anaconda3/envs/htmd/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mhassan/anaconda3/envs/htmd/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt
  File "/home/mhassan/anaconda3/envs/htmd/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mhassan/anaconda3/envs/htmd/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/mhassan/anaconda3/envs/htmd/lib/python3.5/site-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()
  File "/home/mhassan/anaconda3/envs/htmd/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()


KeyboardInterrupt: 

In [52]:
classifier_scores = [gridsearch_rf.best_score_, gridsearch_sv.best_score_] #, gridsearch_nn.best_score_]

In [57]:
best_classifier_index = classifier_scores.index(max(classifier_scores))

In [58]:
import pickle

In [ ]:
result = {'best_classifier': best_classifier_index, 'best_score': classifier_scores[best_classifier_index]}
with open("result_" + pname, "wb") as f:
    pickle.dump(result, f)